# Pre-processing the NICT JLE dataset

## TODO: explanation : regular expressions - used pattern for tags

In [ ]:
import re
from os import listdir


def generateCleanFile(filename):
    input = open("NICT_JLE_4.1/LearnerOriginal/" + filename, "r", encoding="latin1") 
    output = open("preProcessedData/" + filename, "w", encoding="latin1")
    lines = input.readlines()
    for line in lines:
        if '<B>' in line :
            output.write(removeAllInternalTags(line[3:len(line)-5])+"\n")
    input.close()
    output.close() 

def removeAllInternalTags(line):
    pile = list()
    pile = [(m.start(0), m.end(0)) for m in re.finditer(r'<(.*?)>', line)]

    buffer = []
    counter = 0
    while len(pile) != 0:
        if line[pile[counter][0]+1] == '/':
            line = line[0:buffer[-1][0]] + line[pile[counter][1]:]
            pile = [(m.start(0), m.end(0)) for m in re.finditer(r'<(.*?)>', line)]
            counter = 0
            buffer.pop()
        else :
            buffer.append(pile[counter])
            counter+=1
    return line

# Keep only signaficant tags that will help with the prediction of the SST level of each participant 
# tags is the list of tags that the user wants to remove from the transcript 
def removeInternalTags(line, tags=[]):
    for tag in tags:
        doubleTagPattern = "<",tag,">(.+?)</",tag,">"
        singleTagPattern = "</?",tag,"*?>"
        line = re.sub(doubleTagPattern, "", line)
        line = re.sub(singleTagPattern, "", line)
    return re.sub(r"\s+", " ", line)


files = [f for f in listdir("NICT_JLE_4.1/LearnerOriginal")]

#generateCleanFile("file00046.txt")

for f in files:
    generateCleanFile(f)


Processing the dataset: extract features with the Bag of Word

# TODO: explanation : BoW algo - stopwords

In [10]:
import nltk
from nltk.corpus import stopwords
from os import listdir
import numpy
import re


def word_extraction(sentence):    
    ignore = set(stopwords.words('english'))   
    words = re.sub("[^\w]", " ",  sentence).split()    
    cleaned_text = [w.lower() for w in words if w not in ignore]    
    return cleaned_text

def tokenize(sentences):    
    words = []    
    for sentence in sentences:        
        w = word_extraction(sentence)        
        words.extend(w)            
        words = sorted(list(set(words)))    
    return words

def generate_bow(filename):
    f = open("preProcessedData/" + filename, "r", encoding="latin1") 
    lines = f.readlines()
    vocab = tokenize(lines)
    vector = list()        
    for line in lines:        
        words = word_extraction(line)        
        bag_vector = numpy.zeros(len(vocab))        
        for w in words:            
            for i,word in enumerate(vocab):                
                if word == w:                     
                    bag_vector[i] += 1 
    f.close()                 
    return vector

    

files = [f for f in listdir("preProcessedData")]

# TODO : save the BoW output in a variable as our input for the classifier 


# Training the classifier to Predict SST

# Accuracy and Confusion matrix